In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///C:/Users/arunv/nyc_taxi/mlruns/1', creation_time=1721285201813, experiment_id='1', last_update_time=1721285201813, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename, nrows=5000):
    # Read the entire parquet file
    df = pd.read_parquet(filename)

    # Slice the DataFrame to only include the first `nrows` rows
    df = df.head(nrows)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
    




In [8]:
df_train = read_dataframe(filename = './data/green_tripdata_2021-01.parquet')
df_val = read_dataframe(filename = './data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(4684, 4684)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


3.094866326521994

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "arun")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-01.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

  0%|                                                                            | 0/1 [00:00<?, ?trial/s, best loss=?]

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [15:25:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.06240                                                                                            
[1]	validation-rmse:5.67761                                                                                            
[2]	validation-rmse:5.10521                                                                                            
[3]	validation-rmse:4.87344                                                                                            
[4]	validation-rmse:4.77112                                                                                            
[5]	validation-rmse:4.66151                                                                                            
[6]	validation-rmse:4.60849                                                                                            
[7]	validation-rmse:4.57060                                                                                            
[8]	validation-rmse:4.53711             

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|█████████████████████████████████████████████████| 1/1 [02:13<00:00, 133.47s/trial, best loss: 4.0039811560703145]


In [31]:
mlflow.xgboost.autolog(disable=True)
mlflow.end_run()

In [32]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    best_params = {
        'learning_rate':0.42743065855600804,
        'max_depth':31,
        'min_child_weight':2.307341011813763,
        'objective':'reg:linear',
        'reg_alpha':0.3261273025242011,
        'reg_lambda':0.10190692089641412,
        'seed':42  }
    
    mlflow.log_params(best_params)
    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)
    with open("models/preprocessor.b","wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b",artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:04:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:7.06240
[1]	validation-rmse:5.67761
[2]	validation-rmse:5.10521
[3]	validation-rmse:4.87344
[4]	validation-rmse:4.77112
[5]	validation-rmse:4.66151
[6]	validation-rmse:4.60849
[7]	validation-rmse:4.57060
[8]	validation-rmse:4.53711
[9]	validation-rmse:4.52299
[10]	validation-rmse:4.50714
[11]	validation-rmse:4.49955
[12]	validation-rmse:4.48303
[13]	validation-rmse:4.47865
[14]	validation-rmse:4.47402
[15]	validation-rmse:4.46100
[16]	validation-rmse:4.45337
[17]	validation-rmse:4.45064
[18]	validation-rmse:4.44093
[19]	validation-rmse:4.43518
[20]	validation-rmse:4.43066
[21]	validation-rmse:4.42389
[22]	validation-rmse:4.41740
[23]	validation-rmse:4.40807
[24]	validation-rmse:4.40016
[25]	validation-rmse:4.39785
[26]	validation-rmse:4.39319
[27]	validation-rmse:4.38956
[28]	validation-rmse:4.38424
[29]	validation-rmse:4.37756
[30]	validation-rmse:4.37086
[31]	validation-rmse:4.36729
[32]	validation-rmse:4.35982
[33]	validation-rmse:4.35700
[34]	validation-rmse:4.3

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\arunv\nyc_taxi\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:05:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [ ]:
with open('models/xgboost_model.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
params = {
    'learning_rate':0.42743065855600804,
    'max_depth':31,
    'min_child_weight':2.307341011813763,
    'objective':'reg:linear',
    'reg_alpha':0.3261273025242011,
    'reg_lambda':0.10190692089641412,
    'seed':42  }

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
mlflow.log_artifact(local_path="models/xgboost_model.bin", artifact_path="models_pickle")

In [33]:
logged_model = 'runs:/1a52002c34b44032992e9210dc149c10/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



In [34]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 1a52002c34b44032992e9210dc149c10

In [35]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [36]:
xgboost_model

In [37]:
y_pred = xgboost_model.predict(valid)

In [38]:
y_pred[:10]

array([ 3.8610084,  9.232762 ,  5.8901763, 10.08693  ,  2.3301203,
       31.719183 , 23.219332 , 15.26353  ,  9.499435 , 15.567668 ],
      dtype=float32)

In [39]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [41]:
run_id = '1a52002c34b44032992e9210dc149c10'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'C:\\Users\\arunv\\nyc_taxi\\preprocessor'

In [48]:
import pickle
with open("preprocessor/preprocessor.b","rb") as f_in:
    dv = pickle.load(f_in)

DictVectorizer()

[{'PU_DO': '43_151', 'trip_distance': 1.01}, {'PU_DO': '166_239', 'trip_distance': 2.53}, {'PU_DO': '41_42', 'trip_distance': 1.12}, {'PU_DO': '168_75', 'trip_distance': 1.99}, {'PU_DO': '75_75', 'trip_distance': 0.45}]
***


NameError: name 'train_dict' is not defined

In [56]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename, nrows=5000):
    # Read the entire parquet file
    df = pd.read_parquet(filename)

    # Slice the DataFrame to only include the first `nrows` rows
    df = df.head(nrows)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name,stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse":mean_squared_error(y_test,y_pred,squared=False)}
    
    
    
    

In [57]:
df = read_dataframe(filename = './data/green_tripdata_2021-03.parquet')   
X_test = preprocess(df,dv)
target = "duration"
y_test = df[target].values


In [59]:
model_name = "nyc_taxi_regressor"
test_model(name=model_name,stage="Production",X_test=X_test, y_test=y_test)

C:\Users\arunv\nyc_taxi\venv\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
C:\Users\arunv\nyc_taxi\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 4.0039811560703145}